In [ ]:
# !pip install numpy
# !pip install matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math

random.seed(int('0xC0FFEE', 16))
np.random.seed(int('0xC0FFEE', 16))
PI = math.pi

## Numpy data load




In [ ]:
### np.savez('./filename.npz', x=X, y=Y)

practice1_data = np.load('./practice1_data.npz')
print(f'Check key values: {practice1_data.files}')
X, Y = practice1_data['x'], practice1_data['y']
print(X.shape, Y.shape)
print(X[0:10], Y[0:10])

# Practice 1
평균, 시그마가 정해진 두 개의 클래스에서 1차원 데이터를 샘플링해서, 히스토그램과 확률분포 함수를 각각 그려보시오.

In [ ]:
def show_histogram(A, B):
    plt.hist(A, label='a', alpha=0.5)
    plt.hist(B, label='b', alpha=0.5)
    plt.legend(loc='upper left')
    plt.show()

In [ ]:
show_histogram(X[Y==0], X[Y==1])

In [ ]:
def pdf(x, mean, std):
    return (1 / std * np.sqrt(PI * 2)) * np.exp(-(1 / 2) * ((x - mean) / std) * ((x - mean) / std))

In [ ]:
def show_pdf(x, Y_A, Y_B):
    plt.plot(x, Y_A, color='red', label='A')
    plt.plot(x, Y_B, color='blue', label='B')

    plt.legend(loc='upper left')
    plt.xlabel("$x$")
    plt.ylabel("$p(x)$")
    plt.show()

In [ ]:
x = np.linspace(140, 200, 1000)
Y_A = pdf(x, np.mean(X[Y==0]), np.std(X[Y==0]))
Y_B = pdf(x, np.mean(X[Y==1]), np.std(X[Y==1]))

In [ ]:
show_pdf(x, Y_A, Y_B)

# Practice 2

추가적으로 테스트 데이터를 샘플링해서, likelihood값만을 이용하는 경우, prior값과 결합하여 posterior로 볼 경우 각각에 대하여 분류 정확도를 보고하고 

(a) likelihood 함수를 그리고, 

(b) prior 적용 시, likelihood x prior 함수를 그리고,

(c) posterior =  likelihood x prior / p(x) 를 그리시오. 

prior의 변화에 따라 decision point를 그래프에서 대략적으로 읽고, prior에 따라서 분류 결과가 어떻게 변화하는지 설명하시오.

In [ ]:
practice2_train = np.load('./practice2_train.npz')
print(f'Check key values: {practice2_train.files}')
X_train, Y_train = practice2_train['x'], practice2_train['y']
print(X_train.shape)

In [ ]:
practice2_test = np.load('./practice2_test.npz')
print(f'Check key values: {practice2_test.files}')
X_test, Y_test = practice2_test['x'], practice2_test['y']
print(X_test.shape)

In [ ]:
def get_likelihood(x, A_mean, A_std, B_mean, B_std):
    return pdf(x, A_mean, A_std), pdf(x, B_mean, B_std)

In [ ]:
def get_prior(A, B):
    prior_A = len(A)/ (len(A) + len(B))
    return (prior_A), (1 - prior_A)

In [ ]:
def get_posterior(x, A_likelihood, B_likelihood, A_prior, B_prior):
    evidence = (A_likelihood * A_prior) + (B_likelihood * B_prior)
                 
    return A_likelihood * A_prior / evidence, B_likelihood * B_prior / evidence

In [ ]:
def get_accuracy(Y_test, A_prob, B_prob):
    y_pred = np.where(A_prob > B_prob, 0, 1)

    return len(np.where(y_pred == Y_test)[0]) / len(y_pred)

In [ ]:
A_likelihood, B_likelihood = get_likelihood(X_test, 
                                np.mean(X_train[Y_train==0]),
                                np.std(X_train[Y_train==0]),
                                np.mean(X_train[Y_train==1]),
                                np.std(X_train[Y_train==1]))

y_pred = np.where(A_likelihood > B_likelihood, 0, 1)

print(f'Acc: {get_accuracy(Y_test, A_likelihood, B_likelihood)}')

graph_x = np.linspace(140, 200, 1000)
A_graph_x, B_graph_x = get_likelihood(graph_x, 
                                np.mean(X_train[Y_train==0]),
                                np.std(X_train[Y_train==0]),
                                np.mean(X_train[Y_train==1]),
                                np.std(X_train[Y_train==1]))
show_pdf(graph_x, A_graph_x, B_graph_x)

In [ ]:
A_prior, B_prior = get_prior(X_train[Y_train==0], X_train[Y_train==1])

print(A_prior, B_prior)

print(f'Acc: {get_accuracy(Y_test, A_likelihood * A_prior, B_likelihood * B_prior)}')

show_pdf(graph_x, A_graph_x * A_prior, B_graph_x * B_prior)

In [ ]:
A_prob, B_prob = get_posterior(X_test, A_likelihood, B_likelihood, A_prior, B_prior)

print(f'Acc: {get_accuracy(Y_test, A_prob, B_prob)}')

A_graph_prob, B_graph_prob = get_posterior(X_test, A_graph_x, B_graph_x, A_prior, B_prior)


show_pdf(graph_x, A_graph_prob, B_graph_prob)

- prior_A: 0.5
- prior_B: 0.5

In [ ]:
A_prob, B_prob = get_posterior(X_test, A_likelihood, B_likelihood, 0.5, 0.5)

print(f'Acc: {get_accuracy(Y_test, A_prob, B_prob)}')

- prior_A: 0.4
- prior_B: 0.6

In [ ]:
A_prob, B_prob = get_posterior(X_test, A_likelihood, B_likelihood, 0.4, 0.6)

print(f'Acc: {get_accuracy(Y_test, A_prob, B_prob)}')

- prior_A: 0.25
- prior_B: 0.75

In [ ]:
A_prob, B_prob = get_posterior(X_test, A_likelihood, B_likelihood, 0.25, 0.75)

print(f'Acc: {get_accuracy(Y_test, A_prob, B_prob)}')

# Question 1

위에서 prior probability를 0.5:0.5에서 0.25:0.75로 변화시키면서 Bayesian 분류기의 분류 정확도를 평가하였다.
세가지 경우에서 점점 정확도가 떨어지는 이유에 대하여 설명하시오. 

답 :  
테스트 데이터셋의 샘플 ratio는 __0.5:0.5(class 0: class 1)__ 이다.
따라서 특정 클래스의 발생 빈도(클래스의 확률)를 나타내는 prior probability가 0.5:0.5에서 0.25:0.75로 점점 변화된다는 것은, __실제 테스트 데이터셋의 샘플 분포와 점점 달라진다는 것을 의미한다.__  
Prior knowledge가 실제 데이터를 잘 반영하지 못하게 되기 때문에 정확도가 점점 떨어지게 되는 것이다.

In [ ]:
print("Test set ratio: {}:{}".format(len(Y_test[Y_test == 0]) / len(Y_test),len(Y_test[Y_test == 1]) / len(Y_test)))

# Practice 3
두 클래스의 평균 사이의 거리를 점차 증가시키면서 train, test 데이터를 샘플링하여 Bayesian 분류기의 정확도를 계산해보시오.

s = 3
mu = (-1, 1), (-2, 2), (-3, 3)
train / test sampling - report performance

In [ ]:
def get_data(num_data, ratio, A_mean, A_std, B_mean, B_std):

    num_data_A = int(num_data * ratio)
    num_data_B = num_data - num_data_A

    A = np.random.normal(loc=A_mean, scale=A_std, size=num_data_A)
    B = np.random.normal(loc=B_mean, scale=B_std, size=num_data_B)
    label_zeros = np.zeros((num_data_A, 1))
    label_ones = np.ones((num_data_B, 1))
    
    return A, B, label_zeros, label_ones

In [ ]:
# Train data
train_A, train_B, _, _ = get_data(1000, 0.5, -1, 3, 1, 3)
# Test data
test_A, test_B, test_YA, test_YB = get_data(100, 0.5, -1, 3, 1, 3)
test_X = np.concatenate((test_A, test_B))
test_Y = np.concatenate((test_YA, test_YB)).reshape(-1)

In [ ]:
A_likelihood, B_likelihood = get_likelihood(test_X, 
                                np.mean(train_A),
                                np.std(train_A),
                                np.mean(train_B),
                                np.std(train_B))
A_prior, B_prior = get_prior(train_A, train_B)
A_prob, B_prob = get_posterior(test_X, A_likelihood, B_likelihood, A_prior, B_prior)
print(f'Acc: {get_accuracy(test_Y, A_prob, B_prob)}')
show_histogram(train_A, train_B)

In [ ]:
# Train data
train_A, train_B, _, _ = get_data(1000, 0.5, -2, 3, 2, 3)
# Test data
test_A, test_B, test_YA, test_YB = get_data(100, 0.5, -2, 3, 2, 3)
test_X = np.concatenate((test_A, test_B))
test_Y = np.concatenate((test_YA, test_YB)).reshape(-1)

In [ ]:
A_likelihood, B_likelihood = get_likelihood(test_X, 
                                np.mean(train_A),
                                np.std(train_A),
                                np.mean(train_B),
                                np.std(train_B))
A_prior, B_prior = get_prior(train_A, train_B)
A_prob, B_prob = get_posterior(test_X, A_likelihood, B_likelihood, A_prior, B_prior)
print(f'Acc: {get_accuracy(test_Y, A_prob, B_prob)}')
show_histogram(train_A, train_B)

In [ ]:
# Train data
train_A, train_B, _, _ = get_data(1000, 0.5, -3, 3, 3, 3)
# Test data
test_A, test_B, test_YA, test_YB = get_data(100, 0.5, -3, 3, 3, 3)
test_X = np.concatenate((test_A, test_B))
test_Y = np.concatenate((test_YA, test_YB)).reshape(-1)

In [ ]:
A_likelihood, B_likelihood = get_likelihood(test_X, 
                                np.mean(train_A),
                                np.std(train_A),
                                np.mean(train_B),
                                np.std(train_B))
A_prior, B_prior = get_prior(train_A, train_B)
A_prob, B_prob = get_posterior(test_X, A_likelihood, B_likelihood, A_prior, B_prior)
print(f'Acc: {get_accuracy(test_Y, A_prob, B_prob)}')
show_histogram(train_A, train_B)

# Practice 4

Practice 3에서 그린 세 개의 히스토그램을 probability density function으로 그리시오.



In [ ]:
def show_pdf(x, Y_A, Y_B, title=""):
    plt.plot(x, Y_A, color='red', label='A')
    plt.plot(x, Y_B, color='blue', label='B')

    plt.legend(loc='upper left')
    plt.xlabel("$x$")
    plt.ylabel("$p(x)$")
    plt.title("{}".format(title))
    plt.show()

In [ ]:
"""
1. 먼저 Practice 3과 동일한 mean과 standard deviation을 가지는 1000개의 train 데이터셋을 만든다.
2. Train 데이터셋의 min값과 max값을 구하여 그래프를 그릴 x의 range를 구한다.
3. 2에서 구한 x의 범위에 대하여 train_A와 train_B의 mean과 std을 통해 probability density 값을 구한다.
4. 그래프를 그린다.
"""
# Train data set 만들기
train_A, train_B, _, _ = get_data(1000, 0.5, -1, 3, 1, 3)
train_X = np.concatenate((train_A, train_B))

# Train data의 X 범위 구하기
train_min = train_X.min() - 1
train_max = train_X.max() + 1

# 그래프를 그릴 x의 값 구하기(1000개)
graph_x = np.linspace(train_min, train_max, 1000)

# probability density 값 구하기
# mean과 std 값으로 위의 train data set을 만들 때 사용한 값을 그대로 사용해도 되지만,
# 실제 만들어진 data의 값과 약간의 차이가 있으므로 직접 구한 mean과 std를 사용한다.
A_pdf = pdf(graph_x, np.mean(train_A), np.std(train_A))
B_pdf = pdf(graph_x, np.mean(train_B), np.std(train_B))

# 그래프 그리기
show_pdf(graph_x, A_pdf, B_pdf)

In [ ]:
"""
1. 먼저 Practice 3과 동일한 mean과 standard deviation을 가지는 1000개의 train 데이터셋을 만든다.
2. Train 데이터셋의 min값과 max값을 구하여 그래프를 그릴 x의 range를 구한다.
3. 2에서 구한 x의 범위에 대하여 train_A와 train_B의 mean과 std을 통해 probability density 값을 구한다.
4. 그래프를 그린다.
"""
# Train data set 만들기
train_A, train_B, _, _ = get_data(1000, 0.5, -2, 3, 2, 3)
train_X = np.concatenate((train_A, train_B))

# Train data의 X 범위 구하기
train_min = train_X.min() - 1
train_max = train_X.max() + 1

# 그래프를 그릴 x의 값 구하기(1000개)
graph_x = np.linspace(train_min, train_max, 1000)

# probability density 값 구하기
# mean과 std 값으로 위의 train data set을 만들 때 사용한 값을 그대로 사용해도 되지만,
# 실제 만들어진 data의 값과 약간의 차이가 있으므로 직접 구한 mean과 std를 사용한다.
A_pdf = pdf(graph_x, np.mean(train_A), np.std(train_A))
B_pdf = pdf(graph_x, np.mean(train_B), np.std(train_B))

# 그래프 그리기
show_pdf(graph_x, A_pdf, B_pdf)

In [ ]:
"""
1. 먼저 Practice 3과 동일한 mean과 standard deviation을 가지는 1000개의 train 데이터셋을 만든다.
2. Train 데이터셋의 min값과 max값을 구하여 그래프를 그릴 x의 range를 구한다.
3. 2에서 구한 x의 범위에 대하여 train_A와 train_B의 mean과 std을 통해 probability density 값을 구한다.
4. 그래프를 그린다.
"""
# Train data set 만들기
train_A, train_B, _, _ = get_data(1000, 0.5, -3, 3, 3, 3)
train_X = np.concatenate((train_A, train_B))

# Train data의 X 범위 구하기
train_min = train_X.min() - 1
train_max = train_X.max() + 1

# 그래프를 그릴 x의 값 구하기(1000개)
graph_x = np.linspace(train_min, train_max, 1000)

# probability density 값 구하기
# mean과 std 값으로 위의 train data set을 만들 때 사용한 값을 그대로 사용해도 되지만,
# 실제 만들어진 data의 값과 약간의 차이가 있으므로 직접 구한 mean과 std를 사용한다.
A_pdf = pdf(graph_x, np.mean(train_A), np.std(train_A))
B_pdf = pdf(graph_x, np.mean(train_B), np.std(train_B))

# 그래프 그리기
show_pdf(graph_x, A_pdf, B_pdf)

# Practice 5

두 클래스의 데이터가 다음과 같은 Gaussian 분포를 따른다고 할 때, 아래 설명에 따라 Bayesian 분류기의 분류 정확도를 계산하시오. 

A class : mu = -1, s = 3
B class : mu = 3, s = 2

a) P(A)=P(B)일 때, p(x|A), p(x|B), P(A|x), P(B|x)를 그리시오.

b) P(A)=1/3, P(B)=2/3일 때, p(x|A), p(x|B), P(A|x), P(B|x)를 그리시오.

c) a)와 b)에서 구한 p(A|x)=p(B|x)을 풀면 decision point를 구할 수 있다. 각 decision point를 구하여 a)와 b)에서 그린 그래프 위에 표시하시오. 

d) Decision point를 구한 후에 Bayesian 분류기의 분류 방법이 어떻게 간단히 변화할 수 있는지 설명하시오.

e) test set을 A, B 클래스에 대하여 각 100개씩 샘플링하여, d)에서 설명한 방법으로 분류 정확도를 구하시오. 단, P(A)=P(B)



## a) P(A)=P(B)일 때, p(x|A), p(x|B), P(A|x), P(B|x)를 그리시오.

In [ ]:
# 데이터셋 만들기
# A class : mu = -1, s = 3 B class : mu = 3, s = 2
# P(A)=P(B)일 때, P(A)=P(B)=0.5(prior probability)이다.
# get_data 함수를 사용하여 만든다.
train_A, train_B, _, _ = get_data(1000, 0.5, -1, 3, 3, 2)
train_X = np.concatenate((train_A, train_B))

# 위에서 만든 데이터셋의 min과 max값을 사용하여 그래프를 그릴 x의 값(1000개)을 구한다.
train_min = train_X.min() - 10
train_max = train_X.max() + 10
graph_x = np.linspace(train_min, train_max, 1000)

# get_likelihood 함수를 통해 likelihood(p(x|A)와 p(x|B))를 구한다.
A_graph_x, B_graph_x = get_likelihood(graph_x, np.mean(train_A),
                                      np.std(train_A), np.mean(train_B),
                                      np.std(train_B))
# p(x|A)와 p(x|B)를 그린다.
show_pdf(graph_x, A_graph_x, B_graph_x, "p(x|A), p(x|B)")

# get_posterior 함수를 통해 posterior(P(A|x), P(B|x))를 구한다.
A_prior, B_prior = get_prior(train_A, train_B)
A_graph_prob, B_graph_prob = get_posterior(graph_x, A_graph_x, B_graph_x,
                                           A_prior, B_prior)

# P(A|x), P(B|x)를 그린다.
show_pdf(graph_x, A_graph_prob, B_graph_prob, "P(A|x), P(B|x)")

## b) P(A)=1/3, P(B)=2/3일 때, p(x|A), p(x|B), P(A|x), P(B|x)를 그리시오.

In [ ]:
# 데이터셋 만들기
# A class : mu = -1, s = 3 B class : mu = 3, s = 2
# P(A)=1/3, P(B)=2/3
# get_data 함수를 사용하여 만든다.
train_A, train_B, _, _ = get_data(1000, 1 / 3, -1, 3, 3, 2)
train_X = np.concatenate((train_A, train_B))

# 위에서 만든 데이터셋의 min과 max값을 사용하여 그래프를 그릴 x의 값(1000개)을 구한다.
train_min = train_X.min() - 10
train_max = train_X.max() + 10
graph_x = np.linspace(train_min, train_max, 1000)

# get_likelihood 함수를 통해 likelihood(p(x|A)와 p(x|B))를 구한다.
A_graph_x, B_graph_x = get_likelihood(graph_x, np.mean(train_A),
                                      np.std(train_A), np.mean(train_B),
                                      np.std(train_B))
# p(x|A)와 p(x|B)를 그린다.
show_pdf(graph_x, A_graph_x, B_graph_x, "p(x|A), p(x|B)")

# get_posterior 함수를 통해 posterior(P(A|x), P(B|x))를 구한다.
A_prior, B_prior = get_prior(train_A, train_B)
A_graph_prob, B_graph_prob = get_posterior(graph_x, A_graph_x, B_graph_x,
                                           A_prior, B_prior)

# P(A|x), P(B|x)를 그린다.
show_pdf(graph_x, A_graph_prob, B_graph_prob, "P(A|x), P(B|x)")

## c) a)와 b)에서 구한 p(A|x)=p(B|x)을 풀면 decision point를 구할 수 있다. 각 decision point를 구하여 a)와 b)에서 그린 그래프 위에 표시하시오. 

### a)

In [ ]:
def show_pdf_with_decision_point(x,
                                 Y_A,
                                 Y_B,
                                 decision_point_x,
                                 decision_point_y,
                                 title=""):
    plt.plot(x, Y_A, color='red', label='A')
    plt.plot(x, Y_B, color='blue', label='B')
    plt.scatter(decision_point_x,
                decision_point_y,
                color='black',
                label="Decision point")

    plt.legend(loc='upper left')
    plt.xlabel("$x$")
    plt.ylabel("$p(x)$")
    plt.title("{}".format(title))
    plt.show()

In [ ]:
# def get_decision_point(A_mean, A_std, B_mean, B_std):
#     if A_std==B_std:
#         x1 = (A_mean+B_mean)/2
#         x2 = (A_mean+B_mean)/2
#     else:
#         x1 = (-(B_std * B_std * A_mean - A_std * A_std * B_mean) + np.sqrt(
#             (B_std * B_std * A_mean - A_std * A_std * B_mean) *
#             (B_std * B_std * A_mean - A_std * A_std * B_mean) -
#             (A_std * A_std * B_mean * B_mean - B_std * B_std * A_mean * A_mean -
#              2 * A_std * A_std * B_std * B_std - np.log(A_std / B_std) *
#              (A_std * A_std - B_std * B_std)))) / (A_std * A_std - B_std * B_std)
#         x2 = (-(B_std * B_std * A_mean - A_std * A_std * B_mean) - np.sqrt(
#             (B_std * B_std * A_mean - A_std * A_std * B_mean) *
#             (B_std * B_std * A_mean - A_std * A_std * B_mean) -
#             (A_std * A_std * B_mean * B_mean - B_std * B_std * A_mean * A_mean -
#              2 * A_std * A_std * B_std * B_std - np.log(A_std / B_std) *
#              (A_std * A_std - B_std * B_std)))) / (A_std * A_std - B_std * B_std)
#     return x1, x2

In [ ]:
# 데이터셋 만들기
# A class : mu = -1, s = 3 B class : mu = 3, s = 2
# P(A)=P(B)일 때, P(A)=P(B)=0.5(prior probability)이다.
# get_data 함수를 사용하여 만든다.
train_A, train_B, _, _ = get_data(1000, 0.5, -1, 3, 3, 2)
train_X = np.concatenate((train_A, train_B))

# 위에서 만든 데이터셋의 min과 max값을 사용하여 그래프를 그릴 x의 값(1000개)을 구한다.
train_min = train_X.min() - 10
train_max = train_X.max() + 10
graph_x = np.linspace(train_min, train_max, 1000)

# get_likelihood 함수를 통해 likelihood(p(x|A)와 p(x|B))를 구한다.
A_graph_x, B_graph_x = get_likelihood(graph_x, np.mean(train_A),
                                      np.std(train_A), np.mean(train_B),
                                      np.std(train_B))

# get_posterior 함수를 통해 posterior(P(A|x), P(B|x))를 구한다.
A_prior, B_prior = get_prior(train_A, train_B)
A_graph_prob, B_graph_prob = get_posterior(graph_x, A_graph_x, B_graph_x,
                                           A_prior, B_prior)

# decision point를 구한다.
# A_graph_prob와 B_graph_prob이 값들이 point로 이루어져있기 때문에, 현재 생성한 데이터에는 완전히 똑같은 값이 존재하지 않을 수 있다.
# A_graph_prob와 B_graph_prob들을 round 해주어 동일한 값이 나오도록 한다.
# round로 인해 한 지점에서 한 개 이상의 decision point가 생길 수 있다.

# p(A|x)=p(B|x)일 때의 x값과 posterior probability값을 구한다.
decision_point_x = graph_x[np.round_(A_graph_prob, 2) == np.round_(
    B_graph_prob, 2)]  # 소수점 두 자릿수까지 round한다.
# decision_point_y = A_graph_x[np.round_(A_graph_prob,2) == np.round_(B_graph_prob,2)]
decision_point_prob_y = A_graph_prob[np.round_(A_graph_prob, 2) == np.round_(
    B_graph_prob, 2)]

# decision point를 그린다.
# show_pdf_with_decision_point(graph_x, A_graph_x, B_graph_x,decision_point_x, decision_point_y, "p(x|A), p(x|B) with decision point")
show_pdf_with_decision_point(graph_x, A_graph_prob, B_graph_prob,
                             decision_point_x, decision_point_prob_y,
                             "P(A|x), P(B|x) with decision point")

### b)

In [ ]:
# 데이터셋 만들기
# A class : mu = -1, s = 3 B class : mu = 3, s = 2
# P(A)=1/3, P(B)=2/3
# get_data 함수를 사용하여 만든다.
train_A, train_B, _, _ = get_data(1000, 1 / 3, -1, 3, 3, 2)
train_X = np.concatenate((train_A, train_B))

# 위에서 만든 데이터셋의 min과 max값을 사용하여 그래프를 그릴 x의 값(1000개)을 구한다.
train_min = train_X.min() - 10
train_max = train_X.max() + 10
graph_x = np.linspace(train_min, train_max, 1000)

# get_likelihood 함수를 통해 likelihood(p(x|A)와 p(x|B))를 구한다.
A_graph_x, B_graph_x = get_likelihood(graph_x, np.mean(train_A),
                                      np.std(train_A), np.mean(train_B),
                                      np.std(train_B))

# get_posterior 함수를 통해 posterior(P(A|x), P(B|x))를 구한다.
A_prior, B_prior = get_prior(train_A, train_B)
A_graph_prob, B_graph_prob = get_posterior(graph_x, A_graph_x, B_graph_x,
                                           A_prior, B_prior)

# decision point를 구한다.
# A_graph_prob와 B_graph_prob이 값들이 point로 이루어져있기 때문에, 현재 생성한 데이터에는 완전히 똑같은 값이 존재하지 않을 수 있다.
# A_graph_prob와 B_graph_prob들을 round 해주어 동일한 값이 나오도록 한다.
# round로 인해 한 지점에서 한 개 이상의 decision point가 생길 수 있다.

# p(A|x)=p(B|x)일 때의 x값과 posterior probability값을 구한다.
decision_point_x = graph_x[np.round_(A_graph_prob, 2) == np.round_(
    B_graph_prob, 2)]  # 소수점 두 자릿수까지 round한다.
# decision_point_y = A_graph_x[np.round_(A_graph_prob,2) == np.round_(B_graph_prob,2)]
decision_point_prob_y = A_graph_prob[np.round_(A_graph_prob, 2) == np.round_(
    B_graph_prob, 2)]

# decision point를 그린다.
# show_pdf_with_decision_point(graph_x, A_graph_x, B_graph_x,decision_point_x, decision_point_y, "p(x|A), p(x|B) with decision point")
show_pdf_with_decision_point(graph_x, A_graph_prob, B_graph_prob,
                             decision_point_x, decision_point_prob_y,
                             "P(A|x), P(B|x) with decision point")

## d) Decision point를 구한 후에 Bayesian 분류기의 분류 방법이 어떻게 간단히 변화할 수 있는지 설명하시오.

__방법1:__ Decision point를 기준으로 x의 클래스 분류 값이 달라지게 된다. 따라서 decision point를 구한 후, 값들을 기준으로 클래스를 분류할 수 있다. 위 예제에서 첫 번째 decision point 전까지는 class A, 첫 번째 decision point와 두 번째 decision point사이에는 class B, 두 번째 decision point 이후엔 class A로 분류한다.  

__방법2:__ Decision point의 값은 0.5이다. 따라서 posterior probability값이 0.5보다 크면 그 클래스로, 0.5보다 작으면 다른 클래스로 분류하는 방법도 있다.

## e) test set을 A, B 클래스에 대하여 각 100개씩 샘플링하여, d)에서 설명한 방법으로 분류 정확도를 구하시오. 단, P(A)=P(B)

In [ ]:
# 데이터셋 만들기
# A class : mu = -1, s = 3 B class : mu = 3, s = 2
# P(A)=P(B)=0.5
# get_data 함수를 사용하여 만든다.
# Train data
train_A, train_B, _, _ = get_data(1000, 0.5, -1, 3, 3, 2)
# Test data(A, B 클래스에 대하여 각 100개씩, 총 200개)
test_A, test_B, test_YA, test_YB = get_data(200, 0.5, -1, 3, 3, 2)
test_X = np.concatenate((test_A, test_B))
test_Y = np.concatenate((test_YA, test_YB)).reshape(-1)

# posterior probability(P(A|x), P(B|x))를 구한다.
A_likelihood, B_likelihood = get_likelihood(test_X, np.mean(train_A),
                                            np.std(train_A), np.mean(train_B),
                                            np.std(train_B))
A_prior, B_prior = get_prior(train_A, train_B)
A_prob, B_prob = get_posterior(test_X, A_likelihood, B_likelihood, A_prior,
                               B_prior)

# decision point를 구한다.
# A_graph_prob와 B_graph_prob이 값들이 point로 이루어져있기 때문에, 현재 생성한 데이터에는 완전히 똑같은 값이 존재하지 않는다.
# A_graph_prob와 B_graph_prob들을 round 해주어 동일한 값이 나오도록 한다.
# round로 인해 한 개 이상의 decision point가 생길 수 있다.
decision_point = test_X[np.round_(A_prob,
                                  1) == np.round_(B_prob,
                                                  1)]  # 소수점 한 자릿수까지 round한다.

# 방법1: decision point를 기준으로 test_X를 분류한다.
y_pred1 = np.where(test_X < decision_point[0], 0, 1)
accuracy1 = len(np.where(y_pred1 == test_Y)[0]) / len(y_pred1)
print("방법 1 분류 정확도: {}".format(accuracy1))

# 방법2: posterior probability값 0.5을 기준으로, A_prob의 값에 따라 분류한다.
y_pred2 = np.where(A_prob >= 0.5, 0, 1)
accuracy2 = len(np.where(y_pred2 == test_Y)[0]) / len(y_pred2)
print("방법 2 분류 정확도: {}".format(accuracy2))